# Brain parcellation with TorchIO and HighRes3DNet

We are going to perform a full brain parcellation of a 3D T1-weighted MRI using [TorchIO](https://torchio.readthedocs.io/) and a pre-trained [PyTorch](https://pytorch.org/) deep learning model in less than 50 lines of code and less than two minutes.

## TorchIO

TorchIO is a Python package to prepare 3D medical images for deep learning pipelines. Check out the [documentation](https://torchio.readthedocs.io/) and a longer [Colab notebook](https://colab.research.google.com/drive/112NTL8uJXzcMw4PQbUvMQN-WHlVwQS3i) containing many examples.

## HighRes3DNet

HighRes3DNet is a 3D residual network presented by Li et al. in [On the Compactness, Efficiency, and Representation of 3D Convolutional Networks: Brain Parcellation as a Pretext Task](https://link.springer.com/chapter/10.1007/978-3-319-59050-9_28). The authors shared the weights of the model they trained to perform full brain parcellation as in [Geodesic Information Flows: Spatially-Variant Graphs and Their Application to Segmentation and Fusion](https://spiral.imperial.ac.uk/bitstream/10044/1/30755/4/07086081.pdf), also known as GIF parcellation.

The weights were ported from TensorFlow as shown in [my entry to the MICCAI educational challenge 2019](https://github.com/fepegar/miccai-educational-challenge-2019).

First we will install TorchIO and download a couple of useful files:

In [1]:
!pip install torchio
!curl -O https://raw.githubusercontent.com/fepegar/miccai-educational-challenge-2019/master/visualization.py
!curl -O https://raw.githubusercontent.com/fepegar/highresnet/master/GIFNiftyNet.ctbl

     |████████████████████████████████| 92kB 4.0MB/s 
     |████████████████████████████████| 42.5MB 71kB/s 
  Created wheel for Python-Deprecated: filename=Python_Deprecated-1.1.0-py2.py3-none-any.whl size=3069 sha256=68f629bb919cc6079e4e9e79efb8fb0b41a120b0cc2a4d5a1b483198c0babd28
  Stored in directory: /root/.cache/pip/wheels/b8/f1/81/2a12cf4208f219e3f023a6211a47165e74e50d534c6a3c3e64
Successfully built Python-Deprecated
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7380  100  7380    0     0  32227      0 --:--:-- --:--:-- --:--:-- 32368
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7238  100  7238    0     0  26708      0 --:--:-- --:--:-- --:--:-- 26708


In [2]:
from pathlib import Path

import torch
torch.set_grad_enabled(False);
import numpy as np
import torchio as tio
from tqdm.notebook import tqdm
from torchvision.datasets.utils import download_and_extract_archive
import visualization
plot_volume = visualization.plot_volume_interactive
%config InlineBackend.figure_format = 'retina'
print('TorchIO version:', tio.__version__)

If you use TorchIO for your research, please cite the following paper:
Pérez-García et al., TorchIO: a Python library for efficient loading,
preprocessing, augmentation and patch-based sampling of medical images
in deep learning. Link: https://arxiv.org/abs/2003.04696

TorchIO version: 0.17.7


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Preparing the data

Let's download a T1-weighted MRI hosted on the NiftyNet model zoo:

In [3]:
root_dir = Path('data')
download_and_extract_archive(
    'https://github.com/NifTK/NiftyNetModelZoo/raw/5-reorganising-with-lfs/highres3dnet_brain_parcellation/data.tar.gz',
    root_dir
)
mri_path = list(root_dir.glob('*.nii.gz'))[0]
mri_path

Extracting data/data.tar.gz to data


PosixPath('data/OAS1_0145_MR2_mpr_n4_anon_sbj_111.nii.gz')

We will now create an instance of [`torchio.Image`](https://torchio.readthedocs.io/data/dataset.html#image) and pass it to a [`torchio.Subject`](https://torchio.readthedocs.io/data/dataset.html#subject):

In [4]:
subject_oasis = tio.Subject(t1=tio.Image(mri_path, type=tio.INTENSITY))
subject_colin = tio.datasets.Colin27()
subject = subject_oasis  # try subject_colin instead!

Extracting /root/.cache/torchio/mni_colin27_1998_nifti/mni_colin27_1998_nifti.zip to /root/.cache/torchio/mni_colin27_1998_nifti


We are going to apply four [preprocessing transforms](https://torchio.readthedocs.io/transforms/preprocessing.html) to our instance of `Subject`:

1. [`ToCanonical`](https://torchio.readthedocs.io/transforms/preprocessing.html#torchio.transforms.ToCanonical) reorients our image using [NiBabel](https://nipy.org/nibabel/) so that it is in [RAS+ orientation](https://nipy.org/nibabel/image_orientation.html)
2. [`Resample`](https://torchio.readthedocs.io/transforms/preprocessing.html#torchio.transforms.Resample) changes the voxels spacing using [SimpleITK](https://simpleitk.org/). This is analogous to using [`torchvision.transforms.Resize`](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.Resize)
3. [`HistogramStandardization`](https://torchio.readthedocs.io/transforms/preprocessing.html#torchio.transforms.HistogramStandardization) is a sofisticated method based on histogram landmarks training. It was presented by Nyúl in [New Variants of a Method of MRI Scale Standardization](https://pubmed.ncbi.nlm.nih.gov/10784285/)
4. [`ZNormalization`](https://torchio.readthedocs.io/transforms/preprocessing.html#torchio.transforms.ZNormalization) generates an image with zero mean and unit variance. Both normalization transforms use the foreground values (computed as the values over the mean intensity) to calculate the corresponding statistics

In [5]:
# From NiftyNet model zoo
LI_LANDMARKS = "4.4408920985e-16 8.06305571158 15.5085721044 18.7007018006 21.5032879029 26.1413278906 29.9862059045 33.8384058795 38.1891334787 40.7217966068 44.0109152758 58.3906435207 100.0"
li_landmarks = np.array([float(n) for n in LI_LANDMARKS.split()])

transforms = [
    tio.ToCanonical(),
    tio.Resample(1),
    tio.HistogramStandardization(landmarks={'t1': li_landmarks}, masking_method=tio.ZNormalization.mean),
    tio.ZNormalization(masking_method=tio.ZNormalization.mean),
]
transform = tio.Compose(transforms)
preprocessed = transform(subject)

The transforms are chained together using [`torchio.Compose`](https://torchio.readthedocs.io/transforms/augmentation.html#compose) or [`torchvision.transforms.Compose`](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.Compose). As you can see, the interface is similar to [`torchvision.transforms`](https://pytorch.org/docs/stable/torchvision/transforms.html). Let's take a look at the input volume:

In [6]:
plot_volume(subject.t1.data.numpy().squeeze())

Output()

The labels are incorrect because `plot_volume` expects an image in RAS+ orientation. It was a good idea to add `ToCanonical` to our list of transforms:

In [7]:
plot_volume(preprocessed.t1.data.numpy().squeeze())

Output()

## Pretrained model

We will use the wonderful [PyTorch Hub](https://pytorch.org/hub/) to download the pretrained model from GitHub:

In [8]:
repo = 'fepegar/highresnet'
model_name = 'highres3dnet'
model = torch.hub.load(repo, model_name, pretrained=True)
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
print('Device:', device)
model.to(device);

Downloading: "https://github.com/fepegar/highresnet/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/fepegar/highresnet-models/raw/master/highres3dnet_li_parameters-7d297872.pth" to /root/.cache/torch/hub/checkpoints/highres3dnet_li_parameters-7d297872.pth


Device: cuda


## Inference

Since the image is too large for the available GPU, we need to [perform inference using image patches](https://torchio.readthedocs.io/data/patch_based.html).

We will use a [`GridSampler`](https://torchio.readthedocs.io/data/patch_inference.html#grid-sampler) to extract patches from all the necessary locations in the image. `GridSampler` is a subclass of [`torch.utils.data.Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset), so we can easily extract batches of patches using a [`torch.utils.data.Dataloader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataloader).

We will add a small overlap between patches to avoid the border effect.


In [9]:
patch_overlap = 4
patch_size = 128
grid_sampler = tio.inference.GridSampler(
    preprocessed,
    patch_size,
    patch_overlap,
)
patch_loader = torch.utils.data.DataLoader(grid_sampler)
aggregator = tio.inference.GridAggregator(grid_sampler)

model.eval()
for patches_batch in tqdm(patch_loader):
    input_tensor = patches_batch['t1'][tio.DATA].to(device)
    locations = patches_batch[tio.LOCATION]
    logits = model(input_tensor)
    labels = logits.argmax(dim=tio.CHANNELS_DIMENSION, keepdim=True)
    aggregator.add_batch(labels, locations)
output_tensor = aggregator.get_output_tensor()
plot_volume(
    output_tensor.numpy().squeeze(),
    enhance=False,
    colors_path='GIFNiftyNet.ctbl',
)

/usr/local/lib/python3.6/dist-packages/torchio/data/inference/aggregator.py:116: UserWarning: Medical image frameworks such as ITK do not support int64. Casting to int32...
  warnings.warn(message)


Output()

The result is not as good as a full [GIF](https://spiral.imperial.ac.uk/bitstream/10044/1/30755/4/07086081.pdf) or [FreeSurfer](https://surfer.nmr.mgh.harvard.edu/fswiki/recon-all) parcellation but hey, it's 600 times faster!

## Conclusion

We have seen how to combine TorchIO and PyTorch hub to infer a full brain parcellation using patches of a 3D T1-weighted MRI and a pre-trained convolutional neural network.

TorchIO is looking for feedback and contributors. Don't hesitate to [open an issue](https://github.com/fepegar/torchio/issues/new/choose) in the repository with questions or feature requests!